In [1]:
#importing the required libraries
import numpy as np
import pandas as pd
#libraries for plotting 
import seaborn as sns
import matplotlib.pyplot as plt
#libraries for neural network models
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [3]:
##reading input files for FNN model (X)
num_of_RVEs=2000
num_of_descriptors=10
#directory of input files
path=''
## intialize input matrix X
X=np.zeros((num_of_RVEs,num_of_descriptors))
for RVE_num in np.arange(num_of_RVEs):
    X[RVE_num,:]=np.loadtxt(path+'FNN_inputs_RVE_'+str(RVE_num+1)+'.csv',delimiter=',')

In [25]:
##reading output of simulations (y)
y=np.loadtxt(path+'stress_output.csv',delimiter=',')

In [ ]:
##bulding a DataFrame from the input and output data for the purpose of feature engineering
labels='mu_AR std_AR mu_Eq_Dia std_Eq_Dia  ave_num_neighbor coef_uni coef_100 coef_110 coef_std_100 coef_std_110 hw_100 hw_110'
df=pd.DataFrame(X,columns=labels.split())

##add stress results to the dataframe
df['stress']=y

##checking the dataframe
df.describe()

In [ ]:
###Feature engineering
#Correation between stress and input parameters
df.corr()['stress'].sort_values()
corr_plot=pd.DataFrame(df.corr()['stress'].sort_values())
corr_plot=corr_plot.drop(['stress'])
plt.figure(figsize=(12,6))
sns.barplot(x=corr_plot.index, y='stress',data=corr_plot,palette='coolwarm')
plt.savefig('Correlation.jpg')

In [26]:
##preprocessing input data for FNN model; splitting
from sklearn.model_selection import train_test_split
##data splitting
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25)

In [27]:
from sklearn.preprocessing import MinMaxScaler
##input data scaling
scaler = MinMaxScaler()
X_train= scaler.fit_transform(X_train)
X_test= scaler.transform(X_test)

In [28]:
##building the FNN model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam

#from tensorflow.keras.callacks import EarlyStopping

model = Sequential()

model.add(Dense(64,activation='relu'))
model.add(BatchNormalization(center=True, scale=True))
model.add(Dropout(0.3))

model.add(Dense(128,activation='relu'))
model.add(BatchNormalization(center=True, scale=True))
model.add(Dropout(0.4))

model.add(Dense(256,activation='relu'))
model.add(BatchNormalization(center=True, scale=True))
model.add(Dropout(0.3))

model.add(Dense(512,activation='relu'))
model.add(BatchNormalization(center=True, scale=True))
model.add(Dropout(0.4))

model.add(Dense(1024,activation='relu'))
model.add(Dropout(0.3))

model.add(Dense(1024,activation='relu'))
model.add(BatchNormalization(center=True, scale=True))
model.add(Dropout(0.3))

model.add(Dense(128,activation='relu'))
model.add(BatchNormalization(center=True, scale=True))
model.add(Dropout(0.4))

model.add(Dense(1,activation='relu'))
model.compile(optimizer='adam',loss='mse')
model.summary()

In [ ]:
## compiling and training the model

model.fit(x=X_train,y=y_train,
          validation_data=(X_test,y_test),
          epochs=100)

In [ ]:
## Calculating and plotting loss
losses = pd.DataFrame(model.history.history)
losses.plot()

In [ ]:
## Evaluation of the model
from sklearn.metrics import mean_squared_error,mean_absolute_error,explained_variance_score
predictions = model.predict(X_test)
print(mean_absolute_error(y_test,predictions))
print(np.sqrt(mean_squared_error(y_test,predictions)))
print(explained_variance_score(y_test,predictions))

In [ ]:
## plotting performance for evaluation dataset
plt.figure(figsize=(9,9))
plt.rc('xtick', labelsize=16)
plt.rc('ytick', labelsize=16)
plt.xlabel('Actual stress (MPa)', fontsize=20)
plt.ylabel('Predicted stress (MPa)', fontsize=20)
plt.scatter(y_test+400,predictions+400)
plt.plot(y_test+400,y_test+400,'r')
plt.savefig('FNN_uni.jpg')

In [ ]:
## plotting performance for training dataset
plt.figure(figsize=(12,6))
predictions2 = model.predict(X_train)
plt.scatter(y_train,predictions2)
plt.plot(y_train,y_train,'r')